 Florida Irma Streaming LoRA → ControlNet (Depth) → Stable Video Diffusion 

In [ ]:
#@title  Runtime check (GPU strongly recommended)
import torch, sys
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print(" Switch to GPU: Runtime → Change runtime type → GPU")

In [ ]:
#@title  Mount Google Drive (outputs only)
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = "/content/drive/MyDrive/Florida_Irma_Project"
OUT_DIR  = f"{BASE_DIR}/out"
LORA_DIR = f"{BASE_DIR}/lora_irma_fl"
SCR_DIR  = "/dev/shm/irma_stream"   # RAM disk (not persisted)
DEPTH_DIR= f"{BASE_DIR}/control_inputs/depth"

import os, json
for p in [BASE_DIR, OUT_DIR, LORA_DIR, DEPTH_DIR]:
    os.makedirs(p, exist_ok=True)
os.makedirs(SCR_DIR, exist_ok=True)

print(json.dumps({
    "BASE_DIR": BASE_DIR,
    "OUT_DIR": OUT_DIR,
    "LORA_DIR": LORA_DIR,
    "SCR_DIR (RAM)": SCR_DIR,
    "DEPTH_DIR": DEPTH_DIR
}, indent=2))

In [ ]:
#@title  Install dependencies
!pip -q install --upgrade pip
!pip -q install "diffusers>=0.31" transformers accelerate safetensors xformers
!pip -q install sentencepiece ftfy datasets tqdm einops peft
!pip -q install opencv-python imageio imageio-ffmpeg numpy pillow requests beautifulsoup4 humanize

###  Hugging Face authentication
Accept licenses for:
- `runwayml/stable-diffusion-v1-5`
- `stabilityai/stable-video-diffusion-img2vid-xt`
- `lllyasviel/sd-controlnet-depth`

In [ ]:
#@title  Login to Hugging Face (paste your token)
from huggingface_hub import login
HF_TOKEN = ""  #@param {type:"string"}
if HF_TOKEN:
    login(HF_TOKEN)
    print(" Logged in to Hugging Face")
else:
    print(" Paste your HF token in HF_TOKEN and re-run this cell.")

##  Stream/scrape Irma images (kept in RAM only)
We fetch from news galleries (Guardian, Atlantic, Tampa Bay Times), dedupe by content-hash,
resize to 512×512, and save to `/dev/shm/irma_stream` (RAM).

In [ ]:
#@title  Scrape → dedupe → preprocess (512×512) into RAM (/dev/shm)
import os, hashlib, io, requests
from bs4 import BeautifulSoup
from PIL import Image

SCR_DIR = "/dev/shm/irma_stream"
os.makedirs(SCR_DIR, exist_ok=True)

URLS = [
  "https://www.theguardian.com/world/gallery/2017/sep/12/in-pictures-the-aftermath-of-hurricane-irma-in-florida",
  "https://www.theatlantic.com/photo/2017/09/photos-of-the-damage-left-by-hurricane-irma-in-florida/539421/",
  "https://projects.tampabay.com/projects/2017/hurricane-irma/photo-story/irma-damage/"
]

headers = {"User-Agent": "Mozilla/5.0"}
seen_hashes = set()
kept = 0
max_keep = 160

def normalize(u: str):
    if u.startswith("//"): return "https:" + u
    return u

def add_candidate(srcset_or_url, bag: set):
    if not srcset_or_url: return
    s = srcset_or_url.strip()
    if " " in s and "srcset" in srcset_or_url:
        s = s.split(",")[-1].strip().split(" ")[0]
    s = normalize(s)
    if s.startswith("http"):
        bag.add(s)

for page in URLS:
    try:
        html = requests.get(page, headers=headers, timeout=20).text
        soup = BeautifulSoup(html, "html.parser")
        candidates = set()
        for img in soup.find_all("img"):
            for k in ["src", "data-src", "data-original", "data-srcset", "srcset"]:
                add_candidate(img.get(k), candidates)
        for a in soup.find_all("a", href=True):
            h = normalize(a["href"])
            if any(h.lower().endswith(ext) for ext in [".jpg",".jpeg",".png",".webp"]):
                candidates.add(h)
        for url in list(candidates):
            if kept >= max_keep: break
            try:
                r = requests.get(url, headers=headers, timeout=20)
                if r.status_code != 200 or "image" not in r.headers.get("Content-Type",""):
                    continue
                b = r.content
                h = hashlib.sha256(b).hexdigest()
                if h in seen_hashes: 
                    continue
                seen_hashes.add(h)
                im = Image.open(io.BytesIO(b)).convert("RGB")
                im = im.resize((512,512))
                fname = os.path.join(SCR_DIR, f"irma_{h[:16]}.jpg")
                im.save(fname, quality=92)
                kept += 1
            except Exception:
                pass
        if kept >= max_keep: break
    except Exception as e:
        print("Skip:", page, "→", e)

print(f" Collected {kept} images into {SCR_DIR}")

In [ ]:
#@title  Auto-caption scraped images (short documentary prompts)
import os, glob, random, codecs
SCR_DIR = "/dev/shm/irma_stream"
TEMPLATE = [
  "Florida after Hurricane Irma, heavy damage, flooded streets, documentary photo",
  "Post-Irma destruction in the Florida Keys, debris and broken piers, overcast",
  "Hurricane Irma aftermath in Miami, damaged buildings, downed trees, realistic",
  "Naples coastline after Irma, storm surge residue, scattered wreckage, realism"
]
wrote = 0
for img in glob.glob(os.path.join(SCR_DIR, "*.jpg")):
    base, _ = os.path.splitext(img)
    cap = base + ".txt"
    if not os.path.exists(cap):
        with codecs.open(cap, "w", "utf-8") as f:
            f.write(random.choice(TEMPLATE))
        wrote += 1
print(f"Caption stubs written: {wrote}")

In [ ]:
#@title  Fetch LoRA training script (Diffusers)
import os, requests
SCRIPTS = f"{BASE_DIR}/scripts"
os.makedirs(SCRIPTS, exist_ok=True)
url = "https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora.py"
dst = f"{SCRIPTS}/train_text_to_image_lora.py"
if not os.path.exists(dst):
    open(dst,"wb").write(requests.get(url).content)
print("Saved:", dst)

In [ ]:
#@title  Compact LoRA training (~800 steps)
import subprocess
MODEL_NAME = "runwayml/stable-diffusion-v1-5"
DATA_DIR   = "/dev/shm/irma_stream"
OUTPUT_DIR = f"{LORA_DIR}"
cmd = [
    "accelerate","launch","--mixed_precision=fp16",
    f"{BASE_DIR}/scripts/train_text_to_image_lora.py",
    "--pretrained_model_name_or_path", MODEL_NAME,
    "--train_data_dir", DATA_DIR,
    "--caption_column","text",
    "--validation_prompt","A wide shot of post-hurricane Irma destruction in the Florida Keys, flooded streets, broken docks, scattered debris, cloudy sky, documentary realism",
    "--validation_epochs","1000",
    "--checkpointing_steps","400",
    "--seed","42",
    "--resolution","512",
    "--train_batch_size","2",
    "--gradient_accumulation_steps","4",
    "--learning_rate","1e-4",
    "--lr_scheduler","cosine",
    "--lr_warmup_steps","0",
    "--max_train_steps","800",
    "--rank","8",
    "--mixed_precision","fp16",
    "--output_dir",OUTPUT_DIR
]
print(" ".join(cmd))
subprocess.run(cmd, check=False)

In [ ]:
#@title  Build depth map (Drive source if present, else first scraped image)
import os, glob
from PIL import Image
import numpy as np, torch
from transformers import pipeline as hf_pipeline

drive_source = f"{DEPTH_DIR}/source.jpg"
if os.path.exists(drive_source):
    src_path = drive_source
else:
    imgs = sorted(glob.glob("/dev/shm/irma_stream/*.jpg"))
    assert imgs, "No scraped images; re-run scraping."
    src_path = imgs[0]
print("Depth source:", src_path)

depth_estimator = hf_pipeline("depth-estimation", model="Intel/dpt-hybrid-midas")
image = Image.open(src_path).convert("RGB").resize((1024, 576))
with torch.no_grad():
    depth = depth_estimator(image)["depth"]
d = depth.resize(image.size)
import numpy as np
d = np.array(d)
d = (d - d.min()) / (d.max() - d.min() + 1e-8)
d = (d * 255).astype(np.uint8)
depth_img = Image.fromarray(d)
depth_png = f"{DEPTH_DIR}/depth.png"
os.makedirs(DEPTH_DIR, exist_ok=True)
depth_img.save(depth_png)
print("Saved depth map:", depth_png)

In [ ]:
#@title  SD1.5 + ControlNet (Depth) + LoRA → Keyframe
import torch, os
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, DPMSolverMultistepScheduler
from PIL import Image

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-depth", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

try:
    pipe.load_lora_weights(LORA_DIR)
    pipe.fuse_lora()
    print(" LoRA fused.")
except Exception as e:
    print(" LoRA fuse failed:", e)

control_image = Image.open(f"{DEPTH_DIR}/depth.png").convert("L").resize((1024, 576))

prompt = (
  "Documentary photo of post-hurricane Irma destruction in the Florida Keys, "
  "flooded streets, broken docks and boats pushed ashore, scattered debris, "
  "overcast sky, muted colors, realistic, high detail"
)
negative = "people, faces, text, watermark, logo, gore, fire, fantasy, cartoon"

gen = torch.Generator(device="cuda").manual_seed(1234)
image = pipe(
    prompt=prompt,
    negative_prompt=negative,
    image=control_image,
    num_inference_steps=32,
    guidance_scale=7.0,
    controlnet_conditioning_scale=1.0,
    generator=gen,
    height=576, width=1024
).images[0]

import os
os.makedirs(OUT_DIR, exist_ok=True)
keyframe_path = f"{OUT_DIR}/irma_florida_keyframe_controlnet.png"
image.save(keyframe_path)
print("Saved keyframe:", keyframe_path)

In [ ]:
#@title  Stable Video Diffusion (img2vid-xt) → Animate keyframe
import os, torch, imageio, numpy as np
from PIL import Image
from diffusers import StableVideoDiffusionPipeline

KEYFRAME = f"{OUT_DIR}/irma_florida_keyframe_controlnet.png"
assert os.path.exists(KEYFRAME), "Missing keyframe; run previous cell."

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt",
    torch_dtype=torch.float16, variant="fp16"
).to("cuda")

img = Image.open(KEYFRAME).convert("RGB").resize((1024, 576))

motion_bucket_id = 224
noise_aug_strength = 0.02
num_frames = 25
decoder_chunk_size = None

with torch.autocast("cuda"):
    result = pipe(
        img,
        decode_chunk_size=decoder_chunk_size,
        motion_bucket_id=motion_bucket_id,
        noise_aug_strength=noise_aug_strength,
        num_frames=num_frames
    )

frames = result.frames[0]
frames_uint8 = [(f * 255).astype(np.uint8) for f in frames]
mp4_path = f"{OUT_DIR}/irma_florida_svd.mp4"
imageio.mimwrite(mp4_path, frames_uint8, fps=12, quality=8)
print("Wrote video:", mp4_path)

In [ ]:
#@title  List outputs and (optional) clean RAM images
import os, glob, humanize
from pathlib import Path

def list_tree(root):
    for p in sorted(Path(root).rglob("*")):
        if p.is_file():
            sz = humanize.naturalsize(p.stat().st_size, gnu=True)
            print(sz, " - ", p)

print("\n=== LORA OUT ===")
list_tree(LORA_DIR)
print("\n=== OUT ===")
list_tree(OUT_DIR)

CLEAN_RAM = True  #@param {type:"boolean"}
if CLEAN_RAM:
    import shutil
    shutil.rmtree("/dev/shm/irma_stream", ignore_errors=True)
    print("\n RAM dataset cleared from /dev/shm/irma_stream")